In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow
from tensorflow.keras.models import Model
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=20,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.01,
                                  zoom_range=[0.9, 1.25],
                                  brightness_range=[0.5, 1.5],
                                  preprocessing_function = tensorflow.keras.applications.mobilenet.preprocess_input)

In [ ]:
BATCH_SIZE = 30
IMG_ROWS = 224
IMG_COLS = 224
NUM_CLASSES = 43
EPOCHS = 10

In [ ]:
train_path = r"./input/Train"

train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(IMG_ROWS, IMG_COLS),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
mobilenet_model = tensorflow.keras.applications.mobilenet.MobileNet()
outputs = mobilenet_model.layers[-6].output 
predictions = Dense(43, activation='softmax')(outputs)
model = Model(inputs=mobilenet_model.inputs, outputs=predictions)
for layer in model.layers[:-23]:
    layer.trainable = False
model.summary()

In [ ]:
model.compile(
    optimizer=Nadam(lr=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
train_generator.reset()
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=EPOCHS,
                    verbose=1,
                    workers=6
)

In [ ]:
model.save_weights('./MobileNet_Weights.h5')
with open('./MobileNet_Model.json', 'w') as f:
    f.write(model.to_json())

In [ ]:
with open('MobileNet_Model.json', 'r') as f:
    model = model_from_json(f.read())
model.load_weights('MobileNet_Weights.h5')